# Pathway Evaluation
The goal is to analyse the behaviour of the different pathway when they are fed with the chosen substrate Methanol. They are judged by their maximum production and the maximum MA flux in relation to the biomass growth rate.

In [10]:
# Importing used packages and model
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd
import matplotlib.pyplot as plt
model = read_sbml_model('iML1515.xml')
%run -i create_pathway.ipynb
# flux unit is mmol / [gDW h]

In [8]:
# creating the models with the respective pathways

# path_selector legend
# 0 -> no additional reactions
#1 -> nicotinamid pathway  
#2 -> nicotinic acid pathway  
#3 -> nicotinamid riboside pathway

nam_model = create_pathway(model.copy(),1)
na_model = create_pathway(model.copy(),2)
nr_model = create_pathway(model.copy(),3)
models = [(nam_model,'NAMPT'),(na_model,'NMN Synthetase'),(nr_model,'DeoD')]

Added NAM Pathway
Added NA Pathway
Added NR Pathway


In [16]:
medium = model.medium
# create exchange reaction

model.add_boundary(Metabolite (id='ncam_e', compartment = 'e'), type='exchange')

Reaction identifier,EX_ncam_e
Name,exchange
Memory address,0x20fc751bd00
Stoichiometry,ncam_e <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Simulating pathways and visualizing results
The pathways are one by one simulated with the set medium flux. They are analyzed for maximum NMN production in relation to the biomass growth. The phenotypic phase planes show the relation of the carbon yield for malonic acid on methanol as a function of the growth rate.

In [4]:
from cobra.flux_analysis.phenotype_phase_plane import production_envelope
import pandas as pd

model.medium = medium
model.objective = model.reactions.get_by_id('Ex_biomass')

df_yields = pd.DataFrame()
df_max = pd.DataFrame(index=['max carbon yield','max production'])
max_growth = model.slim_optimize()
print('General maximum biomass growth',max_growth)

for pairing in models:
    pathway_model = pairing[0]
    with pathway_model:
        pathway_model.medium = medium
        pathway_model.objective = pathway_model.reactions.get_by_id('Ex_mln')
        ppp = production_envelope(pathway_model,
                    reactions='Ex_biomass',
                    objective='Ex_mln')
        df_yields[pairing[1]]=ppp.carbon_yield_maximum
        df_max[pairing[1]]=[ppp.carbon_yield_maximum.max(),ppp.flux_maximum.max()]

display(df_max)
df_yields['Ex_biomass'] = ppp.Ex_biomass
df_yields.plot(x='Ex_biomass', y=[pairing[1] for pairing in models], title='Pathway car

SyntaxError: EOL while scanning string literal (<ipython-input-4-f7e66beb4c32>, line 25)

### Discussion

# Evaluating optimal O2 uptake